In [1]:
from copy import deepcopy
import numpy as np
import torch
from torch.optim import Adam
import gym
import time
import core
#from spinningup.spinup.utils.logx import EpochLogger

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [55]:
import torch
import torch.nn as nn

In [2]:
import gymnasium as gym
import gym_examples
import envelope

In [3]:
class ReplayBuffer:
    """
    A simple FIFO experience replay buffer for DDPG agents.
    """

    def __init__(self, obs_dim, act_dim, size):
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.obs2_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.done_buf = np.zeros(size, dtype=np.float32)
        self.ptr, self.size, self.max_size = 0, 0, size

    def store(self, obs, act, rew, next_obs, done):
        self.obs_buf[self.ptr] = obs
        self.obs2_buf[self.ptr] = next_obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def sample_batch(self, batch_size=32):
        idxs = np.random.randint(0, self.size, size=batch_size) #batch_size 만큼 ReplayBuffer(D)에서 random sampling
        batch = dict(obs=self.obs_buf[idxs],
                     obs2=self.obs2_buf[idxs],
                     act=self.act_buf[idxs],
                     rew=self.rew_buf[idxs],
                     done=self.done_buf[idxs])
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in batch.items()}

In [4]:
torch.manual_seed(0)
np.random.seed(0)

In [66]:
Fs = 10
N = 500
trans_info = [[{'coeff': 1.0, 'amps': 5 * np.array([-0.0154, -0.011, -0.005414]), 'freqs': [6.48e-3, 5.622e-3, 3.7e-3]},
               {'coeff': 0.01, 'amps': 5 * np.array([0.01237, -0.005816, -0.03928]), 'freqs': [0.01758, 0.005925, 0.002652]}],
              [{'coeff': -0.01, 'amps': 5 * np.array([-0.01568, -0.01486, -0.0064]), 'freqs': [0.00737, 0.0072, -0.0039]},
               {'coeff': 1.0, 'amps': 5 * np.array([-0.0154, -0.011, -0.005414]), 'freqs': [6.48e-3, 5.622e-3, 3.7e-3]}]]
A = envelope.flattop(t0=5.,len=10.,amp=1.,w=1.)
B = envelope.flattop(t0=15.,len=30.,amp=0.8,w=1.)
time = np.arange(N) / Fs  # (ns) 
freq = np.fft.fftfreq(N, d=1/Fs) # (GHz)
initial_state = np.column_stack((A(time).astype('float32'),B(time).astype('float32'))) #이렇게 만들면 ndarray (N,2), dtype=complex 128

<ipython-input-66-8d92774480cf>:11: ComplexWarning: Casting complex values to real discards the imaginary part
  initial_state = np.column_stack((A(time).astype('float32'),B(time).astype('float32'))) #이렇게 만들면 ndarray (N,2), dtype=complex 128


In [7]:
env = gym.make(id='gym_examples/Quantum', Fs=Fs, N=N, initial_state=initial_state,trans_info=trans_info)

/opt/conda/lib/python3.8/site-packages/gymnasium/envs/registration.py:481: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes']
  logger.warn(


In [59]:
obs_dim = env.observation_space.shape
act_dim = env.action_space.shape[0]
print(obs_dim)
print(act_dim)

(500, 2)
500


In [54]:
#Action limit
act_limit = env.action_space.high[0]
act_limit

array([1., 1.], dtype=float32)

In [33]:
ac = core.MLPActorCritic(env.observation_space, env.action_space)

In [38]:
list((256,256))

[256, 256]

In [49]:
pi_sizes = [500]+[256,256]+[500]

In [57]:
pi=core.mlp(pi_sizes, nn.ReLU,nn.Tanh)

In [58]:
pi(initial_state)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray